In [1]:
import numpy as np
import gvar as gv
import sys
sys.path.append('../')
from h5glance import H5Glance 
import os
import lsqfit
import io
import yaml
import warnings  
warnings.simplefilter(action="default")
warnings.filterwarnings('ignore') 

In [2]:
os.chdir('../../nucleon_sigma/xpt')

In [3]:
import fit_analysis as fa 
import fit_routine as fit
import i_o
import priors

In [4]:
from importlib import reload
reload(i_o)
reload(fa)
reload(fit)

<module 'fit_routine' from '/home/gmoney/lqcd/nucleon_sigma/xpt/fit_routine.py'>

In [5]:
from priors import prior
prior = prior
#print(prior)

In [6]:
#gv.load('../data/scale_setting.p')

In [7]:
f = open('models.yaml', 'r')
models = yaml.load(f,Loader=yaml.FullLoader)
    #print(keys)
model_info = {}
for item in models.values():
        #model_info = item['m_p:s_n2lo:d_n4lo:x_n2lo:l_n4lo:lc_t']
        #model_info = item['m_p:s_n2lo:d_n4lo:x_n2lo:l_n4lo:lc_t']
        #model_info = item['m_p:s_nlo:d_nlo:x_nlo:l_nlo:lc_t']
        #model_info = item['m_p:s_n4lo:d_n4lo:x_n4lo:l_n4lo:fpi']
        model_info = item['m_p:s_n4lo:d_n4lo:x_n4lo:l_n4lo:fpi']

        model_info_sigma = item['proton:s_n2lo:d_n4lo:x_n2lo:l_n4lo:lc_f:sigma_t']
        #model_info_div_lam = item['proton:s_n2lo:d_n4lo:x_n2lo:l_n4lo:lc_t']
#print(model_info)

In [8]:
input_output = i_o.InputOutput()
data,ensembles = input_output.get_data(scheme='w0_org',units='Fpi')
prior = input_output.make_prior(data,prior)
phys_point_data = input_output.get_data_phys_point()

In [9]:

fit_div_lam = fit.fit_routine(prior=prior, data=data, model_info=model_info)
fit_div_lam_out = fit_div_lam.fit
print(model_info,fit_div_lam_out)


{'particles': ['proton'], 'eps2a_defn': 'w0_imp', 'order_chiral': 'n2lo', 'order_disc': 'n2lo', 'order_strange': 'n2lo', 'order_light': 'n2lo', 'xpt': False, 'fv': True, 'fit_phys_units': False, 'fit_fpi_units': True, 'sigma': False, 'delta': True, 'axial': True} Least Square Fit:
  chi2/dof [dof] = 10 [17]    Q = 2.8e-28    logGBF = -33.986

Parameters:
     m_{proton,0}        1.233 (83)       [       1.0 (1.0) ]  
     b_{proton,2}         3.51 (58)       [       0.0 (5.0) ]  
     b_{proton,4}         4.1 (3.4)       [       0.0 (5.0) ]  
     d_{proton,a}        -2.59 (23)       [       0.0 (5.0) ]  
    d_{proton,aa}        -0.50 (65)       [       0.0 (5.0) ]  
    d_{proton,al}         8.0 (1.1)       [       0.0 (5.0) ]  *
     d_{proton,s}        -4.88 (12)       [       0.0 (5.0) ]  
    d_{proton,as}        21.26 (67)       [       0.0 (5.0) ]  ****
    d_{proton,ls}         0.19 (62)       [       0.0 (5.0) ]  
    d_{proton,ss}        1.051 (68)       [       0.0 (5.0) ] 

In [10]:
# fit_sigma = fit.fit_routine(prior=prior, data=data, model_info=model_info_sigma)
# fit_sigma_out = fit_sigma.fit
# print(fit_sigma_out.y)

In [11]:
#  fit_test = fit.fit_routine(prior=prior, data=data, model_info=model_info)
#  fit_out = fit_test.fit
#  print(fit_out.format(maxline=True))

In [12]:
from importlib import reload
reload(fa)
fit_analysis = fa.fit_analysis(phys_point_data=phys_point_data, data=data,model_info=model_info, prior=prior)
fit_analysis_sigma = fa.fit_analysis(phys_point_data=phys_point_data, data=data,model_info=model_info_sigma, prior=prior)
extrap_mass = fit_analysis.extrapolated_mass
extrap_mass_deriv = fit_analysis_sigma.extrapolated_mass
m_proton = extrap_mass['proton']
m_proton_deriv =  extrap_mass_deriv['proton']

{'proton': array([154(32), 46.2(8.2), 98(19), 152(32), 171(37), 194(44), 44.4(7.9),
       98(19), 151(32), 172(37), 194(44), 45.4(8.1), 98(19), 155(33),
       152(32), 175(38), 198(45)], dtype=object)}


KeyError: 'fit_phys_units'

In [13]:
reload(io)
m_pi = phys_point_data['m_pi']
lam_chi = phys_point_data['lam_chi']
e_pi = m_pi / lam_chi
sigma = m_pi**2 *gv.deriv(m_proton, m_pi**2)
l3 = gv.gvar('3.53(26)')
l4 = gv.gvar('4.73(10)')
sigma_analytic = e_pi/2 * (1 + e_pi**2*(5/2 - l3/2 - 2*l4 )) *m_proton_deriv
print('sigma w/ gv.deriv:',sigma)
print('sigma analytic:' ,sigma_analytic)


sigma w/ gv.deriv: 700.8(3.1)
sigma analytic: 45.1(1.5)


In [14]:
print(fit_analysis.error_budget)

KeyError: 'm_{proton,0}'